The data for the project is taken from kaggle https://www.kaggle.com/datasets/mostafaabla/garbage-classification.

In the working directory I created a folder data with three empty directories train, val and test.

Run the python notebook sequentially.

[forthebadge](https://forthebadge.com)

In [1]:
import os
import glob
import random
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
!pip install split-folders

In [3]:
import splitfolders

In [4]:
#os.chdir('..')
os.getcwd()
os.listdir()

In [5]:
os.chdir('..')

In [6]:
os.chdir('working')

In [18]:
os.getcwd()
os.chdir('data')

In [8]:
os.mkdir('train')
os.mkdir('test')
os.mkdir('val')

In [9]:
os.chdir('/kaggle/input')
input_folder = r'../input/garbage-classification/garbage_classification'


In [10]:
os.chdir('..')
print(os.getcwd())
os.listdir()

In [20]:
os.chdir('/kaggle/working')
splitfolders.ratio(input_folder, output = 'data', seed = 42, ratio = (.7,.2,.1), group_prefix = None)

In [21]:
train_path = 'data/train'
valid_path = 'data/val'

**Checking the number of images in the training set and validation set**

In [22]:
len(os.listdir(train_path + '/cardboard'))

In [23]:
len(os.listdir(train_path + '/green-glass'))

In [24]:
len(os.listdir(train_path + '/paper'))

In [25]:
len(os.listdir(train_path + '/plastic'))

In [26]:
len(os.listdir(train_path + '/trash'))

In [27]:
len(os.listdir(train_path + '/metal'))

In [28]:
len(os.listdir(valid_path + '/metal'))

**Plotting the images**

In [29]:
## Displaying the first 12 images of metal from the training set

fig = plt.figure(figsize = (15,15))
columns = 4
rows = 3
i = 1

for img_name in os.listdir(train_path+'/metal'):
    img = train_path + '/metal' +'/'+ img_name
    img = plt.imread(img)
    ax = fig.add_subplot(rows, columns, i)
    plt.imshow(img, interpolation = 'nearest')
    ax.title.set_text(img_name)
    i = i+1
    if i == columns*rows+1:
        break
    
plt.show()

In [30]:
## Displaying the first 12 images of plastic from the training set

fig = plt.figure(figsize = (15,15))
columns = 4
rows = 3
i = 1

for img_name in os.listdir(train_path+'/plastic'):
    img = train_path + '/plastic/' + img_name
    img = plt.imread(img)
    ax = fig.add_subplot(rows, columns, i)
    plt.imshow(img, interpolation = 'nearest')
    ax.title.set_text(img_name)
    i = i+1
    if i == columns*rows+1:
        break
    
plt.show()

**Checking the dimension for each image**

In [31]:
images = os.listdir(train_path + '/metal/')

In [32]:
width, height = [], []

for image_name in images:
    img = plt.imread(train_path+ '/metal/' + image_name)
    w, h, c = img.shape
    width.append(w)
    height.append(h)

In [33]:
sns.jointplot( x = width, y = height, kind = 'scatter')
plt.show()

**The images are of varying dimensions, so we will recale to the mean height and width**

In [34]:
import numpy as np

print('Mean width: {}'.format(np.mean(width)))

print('Mean Height: {}'.format(np.mean(height)))

We will keep the size of the image as (298*384 x 3)

In [35]:
IMAGE_SIZE = (298, 384, 3)

**Checking images are scaled or not**

In [36]:
img_name = os.listdir(train_path + '/metal')[0]
img = plt.imread(train_path+'/metal/'+img_name)

print('Maximum pizel value: {}'.format(np.max(img)))

print('Minimum pizel value: {}'.format(np.min(img)))

## Image is not scaled

**Creating Image Data Generator for Training and Validation set**

In [37]:
train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range = 20,
                               shear_range = 0.2,
                               zoom_range = 0.2,
                               horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale = 1./255)

In [38]:
train_batches = train_datagen.flow_from_directory(directory = train_path,
                                                 target_size = (298, 384),
                                                 classes = ['cardboard', 'metal','paper','plastic','trash','green-glass','white-glass','brown-glass','clothes','biological','battery','shoes'],
                                                 batch_size = 10,
                                                 shuffle = True, class_mode = 'categorical')

valid_batches = valid_datagen.flow_from_directory(directory = valid_path,
                                                 target_size = (298, 384),
                                                 classes = ['cardboard', 'metal','paper','plastic','trash','green-glass','white-glass','brown-glass','clothes','biological','battery','shoes'],
                                                 batch_size = 10,
                                                 shuffle = False, class_mode = 'categorical')

In [39]:
train_batches.class_indices

**Let us see how the traing batch looks like**

In [40]:
imgs, labels = next(train_batches) ## getting first 10 images (1st batch)

## Everytime this line of code is run next 10 images are selected

In [41]:
fig, axes = plt.subplots(1, 10, figsize = (20,20))
axes = axes.flatten()
for img, ax in zip(imgs, axes):
    ax.imshow(img)
plt.tight_layout()
plt.show()

In [42]:
labels

In [43]:
img = imgs[0]

print('Maximum pixel value: {}'.format(np.max(img)))

print('Minimum pixel value: {}'.format(np.min(img)))

Note how the image size and maximum and minimum pixel value are changed

Now the images are rescaled!!

**Building CNN model**

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

from tensorflow.keras.callbacks import EarlyStopping

In [45]:
model = Sequential()

model.add(Convolution2D(filters = 128, kernel_size =  (3,3), input_shape = (298,384,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Convolution2D(filters = 128, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Convolution2D(filters = 64, kernel_size = (3,3), activation = 'relu'))
model.add(Convolution2D(filters = 64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Flatten())
model.add(Dense(units = 128 , activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 12 , activation = 'softmax'))

In [46]:
model.compile(optimizer = 'adam' ,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [47]:
model.summary()

In [48]:
r = model.fit(train_batches,
                        steps_per_epoch = len(train_batches),
                        epochs = 10,
                        validation_data = valid_batches,
                        validation_steps = len(valid_batches),
                        callbacks = EarlyStopping(monitor='val_loss', patience = 2))

In [49]:
r.history.keys()

In [50]:
plt.plot(r.history['accuracy'], label = 'Train')
plt.plot(r.history['val_accuracy'], label = 'Test')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [51]:
model.evaluate(train_batches)
#train_batches.classes

In [52]:
model.evaluate(valid_batches)

In [53]:
train_pred = model.predict(train_batches)

In [54]:
train_classes_pred = np.argmax(train_pred, axis = 1)

In [55]:
train_classes_pred

In [56]:
train_classes_actual = train_batches.classes

In [57]:
train_classes_actual

In [58]:
from sklearn.utils.multiclass import type_of_target
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [59]:
valid_classes = model.predict(valid_batches)
valid_classes_pred = np.argmax(valid_classes, axis = 1)
valid_classes_actual = valid_batches.classes
cm = confusion_matrix(valid_classes_actual, valid_classes_pred)
print(train_batches.class_indices)
#print(cm)
print(f"Accuracy: {accuracy_score(valid_classes_actual, valid_classes_pred)}")

In [60]:
sns.color_palette("magma", as_cmap=True)
plt.figure(figsize=(15,15))
ax = sns.heatmap(cm, annot=True, cmap="magma", fmt='d', cbar_kws={'label': 'Scale'})
labels=list(train_batches.class_indices.keys())
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)

In [61]:
model.save('Model11.h5')

In [62]:
from IPython.display import FileLink
FileLink(r'Model11.h5')

In [63]:
pip install gradio

In [67]:
import gradio as gr


from tensorflow.keras.models import load_model
 
# load model
model = load_model('Model11.h5')

classnames = ['battery','biological','brown-glass','cardboard','clothes','green-glass','metal','paper','plastic','shoes','trash','white-glass']



def predict_image(img):
  img_4d=img.reshape(-1,298, 384,3)
  prediction=model.predict(img_4d)[0]
  return {classnames[i]: float(prediction[i]) for i in range(12)}

image = gr.inputs.Image(shape=(298, 384))
label = gr.outputs.Label(num_top_classes=3)




gr.Interface(fn=predict_image, inputs=image,  title="Garbage Classifier",
    description="This is a Garbage Classification Model Trained using Xception Net.Deployed to Hugging Faces using Gradio.",outputs=label,interpretation='default').launch(debug=True,share=True)
